In [1]:
import re
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras import backend as K
import tensorflow as tf
from keras import layers
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, Permute, Flatten, Activation, RepeatVector, Permute, Multiply, Lambda
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

In [2]:
data = pd.read_csv('C:\\Users\\user1\\Documents\\repo\\dialog act RNN\\MRDA\\full_set.csv')
print(data.shape)
data.head(10)

(108201, 6)


,Unnamed: 0,Speaker,Utterances,Basic,General,Full
0,0,fe016,so um,F,fh,fh
1,1,fe016,i was going to try to get out of here like in ...,S,s,rt
2,2,fe016,um,F,fh,fh
3,3,fe016,because i really appreciate people coming.,S,s,s
4,4,fe016,and the main thing that i was going to ask peo...,S,s,s
5,5,fe016,so anything that transcribers or discourse cod...,S,s,e
6,6,fe016,so we have this um,D,fh,fh
7,7,fe016,i think a starting point is clearly the the ch...,S,s,s
8,8,fe016,which don brought a copy of.,S,s,e
9,9,me011,yeah.,B,b,b


In [3]:
data=data.drop(columns=["Unnamed: 0"])
data.head()

,Speaker,Utterances,Basic,General,Full
0,fe016,so um,F,fh,fh
1,fe016,i was going to try to get out of here like in ...,S,s,rt
2,fe016,um,F,fh,fh
3,fe016,because i really appreciate people coming.,S,s,s
4,fe016,and the main thing that i was going to ask peo...,S,s,s


In [4]:
data['Utterances'] = data['Utterances'].apply(lambda x: x.lower())
data['Utterances'] = data['Utterances'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
data['Utterances'].head()

0                                                so um
1    i was going to try to get out of here like in ...
2                                                   um
3            because i really appreciate people coming
4    and the main thing that i was going to ask peo...
Name: Utterances, dtype: object

In [5]:
print((data.groupby('Full')['Full'].agg('count')).size)
data.groupby('Full')['Full'].agg('count')

52


Full
%       3103
2        841
aa      5898
aap      219
am       349
ar       908
arp      150
b      15013
ba      2216
bc        51
bd       387
bh       154
bk      7177
br       236
bs       141
bsc      150
bu      2091
by        11
cc       371
co       674
cs      2662
d       1805
df      3724
e       3200
f        128
fa       259
fe       307
fg      3091
fh      8362
ft       119
fw         6
g         87
h        792
j        463
m        293
na      1112
nd       483
ng       351
no       828
qh       214
qo        74
qr       127
qrr      345
qw       951
qy       669
r        208
rt      3101
s      33472
t        253
t1       198
t3       165
tc       212
Name: Full, dtype: int64

In [6]:
tokenizer = Tokenizer(num_words = 5000, split=" ")
tokenizer.fit_on_texts(data['Utterances'].values)
X = tokenizer.texts_to_sequences(data['Utterances'].values)
X = pad_sequences(X, padding='post')
print(X.shape)
X[:5]

(108201, 77)


array([[  11,   16,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2,   36,   59,    4,  168,    4,   61,   81,   10,   97,   35,
          15,  438,   86,  659,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,

In [7]:
MAX_LENGTH_PER_SENTENCE=77
units = 128
encoder_input = keras.Input(shape=(MAX_LENGTH_PER_SENTENCE))
x =layers.Embedding(input_dim=5000, output_dim=256, input_length=X.shape[1])(encoder_input)
                              
activations = Bidirectional(tf.keras.layers.LSTM(units, dropout=0.3, recurrent_dropout=0.2))(x)

attention = Dense(1, activation='tanh')(activations)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(units*2)(attention)
attention = Permute((2, 1))(attention)

sent_representation = Multiply()([activations, attention])
sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units*2,))(sent_representation)


probabilities = Dense(52, activation='softmax')(sent_representation)


encoder = keras.Model(inputs=[encoder_input], outputs=[probabilities],name='encoder')

In [8]:
encoder.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 77)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 77, 256)      1280000     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 256)          394240      embedding[0][0]                  
__________________________________________________________________________________________________
dense (Dense)                   (None, 1)            257         bidirectional[0][0]              
____________________________________________________________________________________________

In [9]:
y = pd.get_dummies(data['Full']).values
[print(data['Full'][i],y[i]) for i in range (0,5)]

fh [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
rt [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
fh [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
s [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
s [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


[None, None, None, None, None]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
encoder.fit(X_train, y_train, epochs=3, batch_size=32, verbose=1)

Epoch 1/3
2705/2705 [==============================] - 1987s 733ms/step - loss: 1.6499 - accuracy: 0.5610
Epoch 2/3
2705/2705 [==============================] - 2090s 773ms/step - loss: 1.4185 - accuracy: 0.5923
Epoch 3/3
2705/2705 [==============================] - 1513s 559ms/step - loss: 1.3409 - accuracy: 0.6051


In [12]:
score = encoder.evaluate(X_test, y_test) 

677/677 [==============================] - 54s 76ms/step - loss: 1.4302 - accuracy: 0.5930


In [13]:
print("Accuracy:", score[1]*100)
print(X_test[1])
print(y_test[1])

Accuracy: 59.30410027503967
[ 15   1 517   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


In [14]:
a = ["My name is rahul"]
a = tokenizer.texts_to_sequences(a)
a = np.array(a)
a = pad_sequences(a, padding='post', maxlen=77)
prediction = encoder.predict(np.array(a))
print("Shape of example ", a.shape)
print("Shape of predicted ", prediction.shape)
print("Prediction", prediction)
print("Max value", np.max(prediction))
print("Index of the max value " , prediction.argmax(1))

Shape of example  (1, 77)
Shape of predicted  (1, 52)
Prediction [[2.8074702e-02 5.8301650e-03 5.0202999e-03 3.9070328e-03 2.5314579e-03
  6.2030926e-04 5.3803663e-04 8.0874533e-04 3.8110144e-03 5.9513492e-04
  2.5198697e-03 3.0175794e-04 4.9915956e-03 3.4932401e-03 7.2252436e-04
  2.0353943e-03 7.1896911e-03 2.7220356e-04 5.0850320e-03 5.1762760e-03
  1.1268931e-02 4.2823548e-03 7.5719547e-03 7.7434797e-03 8.0090758e-05
  1.9554584e-04 6.8884547e-04 6.0430951e-03 1.3054860e-03 1.7386785e-04
  4.2062500e-05 2.9607411e-04 4.3036358e-04 1.4738229e-02 2.7782931e-03
  3.3114899e-02 4.1778577e-03 6.2481915e-03 1.3458595e-03 2.0051193e-03
  4.5698383e-04 2.0340513e-04 4.8366386e-05 3.2992042e-03 4.6516233e-03
  9.5870728e-03 3.4225080e-02 7.2584814e-01 9.4675757e-03 5.3551826e-03
  1.0572188e-02 8.2300566e-03]]
Max value 0.72584814
Index of the max value  [47]
